# Exploration

In [ ]:
from neo4j import GraphDatabase

driver = GraphDatabase.driver('bolt://0.0.0.0:7687',
                              auth=('neo4j', 'neo4j'))

- **Quel est le nombre de correspondances par station ?**

In [ ]:
query = '''
MATCH (s:Station)-[c:CORRESPONDANCE]->(s2:Station)
RETURN s.name AS Stations, collect(distinct(s2.ligne)) AS Correspondances, count(distinct(s2)) AS `Nombre de correspondances` ORDER BY Stations
'''

with driver.session() as session:
    print(query)
    session.run(query)

- **Quel est le nombre de stations à moins de deux kilomètres de la station LADEFENSE (on pourra prendre la distance brute sans considération de relation) ?**

In [ ]:
query = '''
MATCH (s1:Station {name_clean: 'LADEFENSE'})-[d:DISTANCE]->(s2:Station)
WHERE d.distance_km < 2 AND d.distance_km > 0
RETURN collect(s2.name) AS Station, count(s2)
'''

with driver.session() as session:
    print(query)
    session.run(query)

- **Combien de temps faut-il pour aller en metro de LADEFENSE à CHATEAUDEVINCENNES ?**

Note : On compte environ 30 secondes d'arrêt par station (moins celle de départ et d'arrivée)

In [ ]:
query = '''
MATCH (s1:Station {ligne: '1'})-[l:LIAISON_METRO {ligne: '1'}]-(s2:Station {ligne: '1'})
RETURN sum(distinct(l.time_minutes)) + 0.5 * (count(distinct(s1)) - 2)
'''

with driver.session() as session:
    print(query)
    session.run(query)

- **Combien de temps faut-il pour aller à pied de LADEFENSE à CHATEAUDEVINCENNES (on pourra considérer que tout le chemin se fait à pied, sans considération de relation) ?**

In [ ]:
query = '''
MATCH (s1:Station {name_clean: 'LADEFENSE'})-[d:DISTANCE]->(s2:Station {name_clean: 'CHATEAUDEVINCENNES'})
RETURN d.distance_km / 4
'''

with driver.session() as session:
    print(query)
    session.run(query)

- **Est-il plus rapide de faire un changement à SAINTLAZARE pour aller de MONTPARNASSEBIENVENUE à GABRIELPERI ?**

Il faut prendre en compte s1.ligne = s2.ligne car sinon l'algorithme ne prend pas en compte les correspondances. Il n'est pas plus rapide de faire un changement à Saint Lazare.

In [ ]:
query = '''
MATCH (start:Station {name_clean: 'MONTPARNASSEBIENVENUE'})
MATCH (end:Station {name_clean: 'GABRIELPERI'})
CALL gds.alpha.shortestPath.stream({
  nodeQuery: 'MATCH (n) RETURN id(n) as id',
  relationshipQuery: 'MATCH (n1)-[r]-(n2) WHERE n1.ligne = n2.ligne RETURN id(r) as id, id(n1) as source, id(n2) as target, r.time_minutes as weight',
  startNode: start,
  endNode: end,
  relationshipWeightProperty: 'weight'
})
YIELD nodeId, cost
RETURN gds.util.asNode(nodeId).name AS Station, gds.util.asNode(nodeId).ligne AS Ligne, cost
'''

with driver.session() as session:
    print(query)
    session.run(query)

- **Combien de stations se trouvent dans un rayon de 10 stations par train autour de STLAZARE ?**

In [ ]:
query = '''
MATCH (n1 {name_clean: "STLAZARE"})
MATCH (n1)-[:LIAISON_METRO*..10]-(n2)
RETURN collect(distinct(n2.name)), count(distinct(n2.name))
'''

with driver.session() as session:
    print(query)
    session.run(query)

- **Combien de stations se trouvent dans un rayon de 20 minutes par train autour de STLAZARE ?**

Normalement la requête est la suivante, mais elle est trop longue pour que je voie les résultats.
- On matche les chemins de type LIAISON_METRO de n'importe quelle longueur et allant de Saint-Lazare à une autre station.
- On défait la liste de relations avec UNWIND.
- On calcule la somme des time_minutes dans chaque chemin.
- On filtre le chemins dont la somme est inférieure à 20 minutes


In [ ]:
query = '''
MATCH path = (s1 {name_clean: "STLAZARE"})-[:LIAISON_METRO*]-(s2:Station)
WHERE s1.name <> s2.name
UNWIND relationships(path) AS rel
WITH *, path, SUM(rel.time_minutes) AS sum
WHERE sum > 1
RETURN count(distinct(s2.name)), collect(distinct(s2.name))
'''

with driver.session() as session:
    print(query)
    session.run(query)

Avec une requête moins couteuse: on prend uniquement les gares situées à moins 4 stations et plus de 2 minutes de Saint-Lazare

In [ ]:
'''
MATCH path = (s1 {name_clean: "STLAZARE"})-[:LIAISON_METRO*..4]-(s2:Station)
WHERE s1.name <> s2.name
UNWIND relationships(path) AS rel
WITH *, path, SUM(rel.time_minutes) AS sum
WHERE sum < 2
RETURN count(distinct(s2.name)), collect(distinct(s2.name))
'''